# Data Sources

In [ ]:
import requests 
import pandas as pd
import json
from datetime import datetime, timedelta
import math

In [2]:
with open('keys.json', 'r') as KeysFile:
    data = json.load(KeysFile)

aviationStackAPIKey = data["Aviation Stack API KEY"]

## Country Data

In [3]:
def getCountryCodes(country: str):
    urlCountry = f"https://api.aviationstack.com/v1/countries?access_key={aviationStackAPIKey}"
    queryStringCountry = {"search": country}
    responseCountry = requests.get(urlCountry, params=queryStringCountry)
    responseCountry = responseCountry.json()
    responseCountry = responseCountry.get('data')
    codes = [c.get('country_iso2') for c in responseCountry]
    return codes

In [57]:
def getAirportsByCountry(code: str):
    urlAirport = f"https://api.aviationstack.com/v1/airports?access_key={aviationStackAPIKey}"
    
    offset = 0
    all_data = []
    
    while True:
        queryStringAirport = {"offset": offset, "country_iso": code, "type": "international"}
        responseAirport = requests.get(urlAirport, params=queryStringAirport)
        responseAirport = responseAirport.json()
        
        currentData = responseAirport.get('data', [])
        currentData = [airP for airP in currentData if airP.get('country_iso2') == code]
        all_data.extend(currentData)
        
        if offset + 100 >= responseAirport.get('pagination').get('total'):
            break
        
        offset += 100
    
    airports_df = pd.DataFrame(columns=['IATA', 'CountryCode', 'AirportName', 'Country'])
    
    for case in all_data:
        new_case = {'IATA': case.get('iata_code', ''),
                    'CountryCode': case.get('country_iso2', ''),
                    'AirportName': case.get('airport_name', ''),
                    'Country': case.get('country_name', '')}
        airports_df.loc[len(airports_df)] = new_case
        
    return airports_df


## Historical Flight Data

In [76]:
def getHistoricalFlights(departureIATA: str=None, arrivalIATA: str=None, date: str=None):
    url = f"https://api.aviationstack.com/v1/flights?access_key={aviationStackAPIKey}"
    queryString = {"limit": 100, "dep_iata": departureIATA, "arr_iata": arrivalIATA, "flight_date": date}
    queryString = {key: value for key, value in queryString.items() if value is not None}
    print(queryString)
    response = requests.get(url, params=queryString)
    if response.status_code != 200:
        print(response)
        return None

    response = response.json()
    response = response['data']
    flights_df = pd.DataFrame(columns=['Date', 'DepartureIATA', 'DepartureTime', 'ArrivalIATA', 'ArrivalTime', 'Aircraft', 'AirlineName', 'AirlineIATA', 'FlightNumberIATA'])
    for flight in response:
        if isinstance(flight, dict) and flight.get('flight', {}).get('codeshared') is None:
            new_case = {
                'Date': date,
                'DepartureIATA': flight.get('departure', {}).get('iata', None),
                'DepartureTime': flight.get('departure', {}).get('scheduled', None),
                'ArrivalIATA': flight.get('arrival', {}).get('iata', None),
                'ArrivalTime': flight.get('arrival', {}).get('scheduled', None),
                'Aircraft': flight.get('aircraft', {}).get('iata', None) if flight.get('aircraft') is not None else None,
                'AirlineName': flight.get('airline', {}).get('name', None),
                'AirlineIATA': flight.get('airline', {}).get('iata', None),
                'FlightNumberIATA': flight.get('flight', {}).get('iata', None)
            }
            flights_df.loc[len(flights_df)] = new_case

    return flights_df

### Get Country Codes for US Brazil Portugal Canada Italy France

In [14]:
US_codes = getCountryCodes('United States')
Brazil_codes = getCountryCodes('Brazil')
Portugal_codes = getCountryCodes('Portugal')
Canada_codes = getCountryCodes('Canada')
Italy_codes = getCountryCodes('Italy')
France_codes = getCountryCodes('France')

In [15]:
print(f"Country codes:\nUS: {US_codes}\nBrazil: {Brazil_codes}\nPortugal: {Portugal_codes}\nCanada: {Canada_codes}\nItaly: {Italy_codes}\nFrance: {France_codes}")

Country codes:
US: ['UM', 'US']
Brazil: ['BR']
Portugal: ['PT']
Canada: ['CA']
Italy: ['IT']
France: ['FR', 'MQ']


In [ ]:
# DO NOT RUN 
US_airports = getAirportsByCountry('US')['IATA']
Brazil_airports = getAirportsByCountry('BR')['IATA']
Portugal_airports = getAirportsByCountry('PT')['IATA']
Canada_airports = getAirportsByCountry('CA')['IATA']
Italy_airports = getAirportsByCountry('IT')['IATA']
France_airports = getAirportsByCountry('FR')['IATA']
# Save dataframes as csv to minimize API requests
US_airports.to_csv(r'Data/Airports/US_airports.csv')
Brazil_airports.to_csv(r'Data/Airports/Brazil_airports.csv')
Portugal_airports.to_csv(r'Data/Airports/Portugal_airports.csv')
Canada_airports.to_csv(r'Data/Airports/Canada_airports.csv')
Italy_airports.to_csv(r'Data/Airports/Italy_airports.csv')
France_airports.to_csv(r'Data/Airports/France_airports.csv')

In [70]:
US_airports = pd.read_csv(r'Data/Airports/US_airports.csv')
Brazil_airports = pd.read_csv(r'Data/Airports/Brazil_airports.csv')
Portugal_airports = pd.read_csv(r'Data/Airports/Portugal_airports.csv')
Canada_airports = pd.read_csv(r'Data/Airports/Canada_airports.csv')
Italy_airports = pd.read_csv(r'Data/Airports/Italy_airports.csv')
France_airports = pd.read_csv(r'Data/Airports/France_airports.csv')

In [77]:
df = getHistoricalFlights(arrivalIATA="FCO", date='2024-12-01')

{'limit': 100, 'arr_iata': 'FCO', 'flight_date': '2024-12-01'}


In [78]:
df

,Date,DepartureIATA,DepartureTime,ArrivalIATA,ArrivalTime,Aircraft,AirlineName,AirlineIATA,FlightNumberIATA
0,2024-12-01,VIE,2024-12-01T06:50:00+00:00,FCO,2024-12-01T08:30:00+00:00,None,Lauda Europe,LW,LW1645
1,2024-12-01,TLV,2024-12-01T18:35:00+00:00,FCO,2024-12-01T21:15:00+00:00,None,SkyUp Airlines,PQ,PQ335
2,2024-12-01,BRU,2024-12-01T18:20:00+00:00,FCO,2024-12-01T20:25:00+00:00,A319,Brussels Airlines,SN,SN3183
3,2024-12-01,BRU,2024-12-01T18:25:00+00:00,FCO,2024-12-01T20:30:00+00:00,A20N,ITA Airways,AZ,AZ165
4,2024-12-01,CPH,2024-12-01T09:45:00+00:00,FCO,2024-12-01T12:20:00+00:00,B38M,Ryanair,FR,FR406
5,2024-12-01,LIN,2024-12-01T11:00:00+00:00,FCO,2024-12-01T12:10:00+00:00,A319,ITA Airways,AZ,AZ2045
6,2024-12-01,OPO,2024-12-01T06:15:00+00:00,FCO,2024-12-01T10:10:00+00:00,B738,Ryanair,FR,FR1191
7,2024-12-01,CMN,2024-12-01T13:00:00+00:00,FCO,2024-12-01T16:10:00+00:00,B738,Royal Air Maroc,AT,AT940
8,2024-12-01,DOH,2024-12-01T09:25:00+00:00,FCO,2024-12-01T13:30:00+00:00,B788,Qatar Airways,QR,QR131
9,2024-12-01,JED,2024-12-01T19:10:00+00:00,FCO,2024-12-01T22:35:00+00:00,None,Wizz Air,W6,W66230


In [9]:
dates = [datetime(2024, 1, 1) + timedelta(days=i) for i in range(366)]

## Future Flights

In [8]:
def getFutureFlights(departureIATA: str=None, arrivalIATA: str=None, date: str=None, airlineIATA: str=None, flightNumber: str=None):
    url = f"https://api.aviationstack.com/v1/flightsFuture?access_key={aviationStackAPIKey}"
    if not ((departureIATA is None) ^ (arrivalIATA is None)):
        return "Invalid Request"
    
    if departureIATA:
        queryString = {
            "iataCode": departureIATA,
            "type": "departure",
            "date": date,
            "airline_iata": airlineIATA,
            "flight_number":flightNumber
        }
    
    else:
        queryString = {
            "iataCode": arrivalIATA,
            "type": "arrival",
            "date": date,
            "airline_iata": airlineIATA,
            "flight_number":flightNumber
        }
    
    queryString = {key: value for key, value in queryString.items() if value is not None}
    response = requests.get(url, params=queryString)
    if response.status_code != 200:
        print(response)
        return None
    response = response.json()
    response = response['data']
    response = [flight for flight in response if "codeshared" not in flight.keys()]
    df = pd.DataFrame(columns=['Date', 'Weekday', 'DepartureIATA', 'DepartureTime', 'ArrivalIATA', 'ArrivalTime', 'Aircraft', 'AirlineIATA', 'FlightNumberIATA'])
    
    for flight in response:
        new_case = {'Date': date, 
                    'Weekday': flight['weekday'],
                    'DepartureIATA': flight['departure']['iataCode'],
                    'DepartureTime': flight['departure']['scheduledTime'],
                    'ArrivalIATA': flight['arrival']['iataCode'],
                    'ArrivalTime': flight['arrival']['scheduledTime'],
                    'Aircraft': flight['aircraft']['modelCode'],
                    'AirlineIATA': flight['airline']['iataCode'],
                    'FlightNumberIATA': flight['flight']['iataNumber']}
        df.loc[len(df)] = new_case

    return df

In [ ]:
getFutureFlights(departureIATA="ATL", date="2024-11-29", airlineIATA="DL")